In [1]:
import sys
PATH_TO_PROJECT = "/home/tanawatp/Documents/Hexapod/PhysicalAI_modelling"
sys.path.append(PATH_TO_PROJECT)
from UtilsFunction.UtilsFunc import*
from UtilsFunction.KinematicsFunc import*
from pydrake.geometry import StartMeshcat
from pydrake.visualization import ModelVisualizer
from pydrake.math import RigidTransform, RotationMatrix

In [2]:
plant,robot_ind = DefinePlant(model_name="Hexapod_mockup",
                    is_fixed=True,
                    fixed_frame="BaseLink")
# Start Meshcat
meshcat = StartMeshcat()
meshcat.SetProperty("/drake/visualizer", "visible", False)

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [3]:
P = CalcfForwaredKinematics(plant=plant,
                            joint_angles=np.zeros(18),
                            base_frame_name="BaseLink",
                            end_effector_frame_name="middle_left_Endeffector")
print(P)

[-6.78383573e-05  3.48999990e-01 -1.66920000e-01]


In [4]:
ListFrameNames(plant=plant,model_instance_index=robot_ind)

Robot frames name
FrameIndex(1) -> BaseLink
FrameIndex(2) -> base_x_axis
FrameIndex(3) -> base_z_axis
FrameIndex(4) -> front_left_HeelLink
FrameIndex(5) -> front_left_HipLink
FrameIndex(6) -> front_left_KneeLink
FrameIndex(7) -> front_left_Endeffector
FrameIndex(8) -> middle_left_HeelLink
FrameIndex(9) -> middle_left_HipLink
FrameIndex(10) -> middle_left_KneeLink
FrameIndex(11) -> middle_left_Endeffector
FrameIndex(12) -> rear_left_HeelLink
FrameIndex(13) -> rear_left_HipLink
FrameIndex(14) -> rear_left_KneeLink
FrameIndex(15) -> rear_left_Endeffector
FrameIndex(16) -> front_right_HeelLink
FrameIndex(17) -> front_right_HipLink
FrameIndex(18) -> front_right_KneeLink
FrameIndex(19) -> front_right_Endeffector
FrameIndex(20) -> middle_right_HeelLink
FrameIndex(21) -> middle_right_HipLink
FrameIndex(22) -> middle_right_KneeLink
FrameIndex(23) -> middle_right_Endeffector
FrameIndex(24) -> rear_right_HeelLink
FrameIndex(25) -> rear_right_HipLink
FrameIndex(26) -> rear_right_KneeLink
FrameInde

In [5]:
IK_solver = InverseKinematics(plant)
end_effector_names = ["middle_left_Endeffector",
                      "front_left_Endeffector",
                      "front_right_Endeffector",
                      "middle_right_Endeffector",
                      "rear_right_Endeffector",
                      "rear_left_Endeffector"]
hip_offset_angle = [0, 
                    -np.pi/3.0, 
                    -2.0*np.pi/3.0, 
                    -np.pi, 
                    -4.0*np.pi/3.0, 
                    -5.0*np.pi/3.0]
base_frame = plant.GetFrameByName("BaseLink")
desired_base_heigh = 0.1
desired_leg_lenght = 0.35

# Add position constraints
for i in range(6):
    end_effector_frame = plant.GetFrameByName(end_effector_names[i])
    print(end_effector_names[i])
    eff_P_base = RotationMatrix.MakeZRotation(hip_offset_angle[i]) @ np.array([0.0,desired_leg_lenght,-1.0*desired_base_heigh])
    print(eff_P_base)
    IK_solver.AddPositionConstraint(
        frameB=end_effector_frame,
        p_BQ=np.zeros(3),     # point in end-effector frame
        frameA=base_frame,
        p_AQ_lower=eff_P_base-0.001,
        p_AQ_upper=eff_P_base+0.001
    )
q_variables = IK_solver.q()
# for i in range(plant.num_positions()):
# for i in [0,3,6,9,12,15]:
#     IK_solver.prog().AddBoundingBoxConstraint(-2.0*np.pi/3.0, 2.0*np.pi/3.0, q_variables[i])
initial_guess = np.ones(plant.num_positions())*0.1
result = Solve(IK_solver.prog(),initial_guess)
q = None
if result.is_success():
    q_solution = result.GetSolution(q_variables)
    print(q_solution)
    q = q_solution

middle_left_Endeffector
[ 0.    0.35 -0.1 ]
front_left_Endeffector
[ 0.30310889  0.175      -0.1       ]
front_right_Endeffector
[ 0.30310889 -0.175      -0.1       ]
middle_right_Endeffector
[ 4.2862638e-17 -3.5000000e-01 -1.0000000e-01]
rear_right_Endeffector
[-0.30310889 -0.175      -0.1       ]
rear_left_Endeffector
[-0.30310889  0.175      -0.1       ]
[-0.00158546 -0.61015183  0.31768941 -0.0020468  -0.60865701  0.31556267
  0.00158862 -0.61015187  0.31768942  0.00158862 -0.61015187  0.31768942
 -0.00147894 -0.60865698  0.31556261 -0.00158546 -0.61015183  0.31768941]


In [6]:
# Create ModelVisualizer and add robot
visualizer = ModelVisualizer(meshcat=meshcat,
                             visualize_frames=True,
                             triad_length=0.05,
                             triad_radius=0.001)
urdf_name = "Hexapod_mockup.urdf"
robot_urdf_path = f"{PATH_TO_PROJECT}/Models/{urdf_name}"
robot = visualizer.parser().AddModels(robot_urdf_path)
plant_viz = visualizer.parser().plant()
robot_body_frame = plant_viz.GetFrameByName("BaseLink")
init_pos = [0.0,0.0,0.0] 
init_orien = np.asarray([0, 0, 0])
X_floorRobot = RigidTransform(
    RollPitchYaw(init_orien * np.pi / 180), p=[0.0,0.0,0.0])
plant_viz.WeldFrames(plant_viz.world_frame(),robot_body_frame, X_floorRobot)
visualizer.Finalize()
# Create context
context = plant_viz.CreateDefaultContext()
if result.is_success():
    visualizer.Run(position=q,loop_once=False)
else:
    visualizer.Run(loop_once=False)


INFO:drake:Click 'Stop Running' or press Esc to quit
